In [3]:
%sx spec2nii twix -v meas_MID00113_FID127014_mega_press_TR2000_V1_.dat

['',
 'Scan 1/2, read all mdhs:   0%|          | 0.00/33.2M [00:00<?, ?B/s]',
 '                                                                    ',
 '',
 'Scan 2/2, read all mdhs:   0%|          | 0.00/265M [00:00<?, ?B/s]',
 'Scan 2/2, read all mdhs:  60%|██████    | 160M/265M [00:00<00:00, 1.68GB/s]',
 '                                                                           ',
 'pymapVBVD version 0.4.2',
 'Software version: VD',
 'Contents of file: meas_MID00113_FID127014_mega_press_TR2000_V1_.dat',
 'Multiraid file, 2 files found.',
 'Selecting file 2. Use -m option to change.',
 'The file contains these evalinfo flags with dimensions and sizes as follows:',
 'image          :\tCol, Cha, Eco, Set  \t[4128   32    2  128]',
 'noise          :\tCol, Cha            \t[4128   32]',
 'phasecor       :\tCol, Cha            \t[4128   32]']

In [25]:
%sx spec2nii twix -q -o scan_1_68ms_2s/raw_nifti -f metab_raw -e image meas_MID00113_FID127014_mega_press_TR2000_V1_.dat

['',
 'read data:  99%|███████████████████████████▊| 254/256 [00:01<00:00, 219.95it/s]',
 '                                                                               ',
 'Header extension validated!']

In [24]:
%sx spec2nii twix -q -o scan_1_68ms_2s/raw_nifti -f wref_internal -e phasecor meas_MID00113_FID127014_mega_press_TR2000_V1_.dat
%sx spec2nii twix -q -o scan_1_68ms_2s/raw_nifti -f wref_quant -e image meas_MID00117_FID127018_mega_press_wref3_V1.dat

['',
 'read data:   0%|                                         | 0/2 [00:00<?, ?it/s]',
 '                                                                               ',
 'Header extension validated!']

In [26]:
%sx mrs_tools info scan_1_68ms_2s/raw_nifti/metab_raw.nii.gz

['',
 'Read file metab_raw.nii.gz (/Users/wclarke/Documents/Data/HelpingPeople/BI/mmsuppresed_170621/scan_1_68ms_2s/raw_nifti).',
 'NIfTI-MRS version 0.2',
 'Data shape (1, 1, 1, 4128, 32, 2, 128)',
 "Dimension tags: ['DIM_COIL', 'DIM_EDIT', 'DIM_DYN']",
 'Spectrometer Frequency: 123.255795 MHz',
 'Dwelltime (Bandwidth): 1.250E-04s (8000 Hz)',
 'Nucleus: 1H',
 'Field Strength: 2.89 T',
 '']

In [27]:
%sx mrs_tools info scan_1_68ms_2s/raw_nifti/wref_internal.nii.gz

['',
 'Read file wref_internal.nii.gz (/Users/wclarke/Documents/Data/HelpingPeople/BI/mmsuppresed_170621/scan_1_68ms_2s/raw_nifti).',
 'NIfTI-MRS version 0.2',
 'Data shape (1, 1, 1, 4128, 32)',
 "Dimension tags: ['DIM_COIL', None, None]",
 'Spectrometer Frequency: 123.255795 MHz',
 'Dwelltime (Bandwidth): 1.250E-04s (8000 Hz)',
 'Nucleus: 1H',
 'Field Strength: 2.89 T',
 '']

In [28]:
%sx mrs_tools info scan_1_68ms_2s/raw_nifti/wref_quant.nii.gz

['',
 'Read file wref_quant.nii.gz (/Users/wclarke/Documents/Data/HelpingPeople/BI/mmsuppresed_170621/scan_1_68ms_2s/raw_nifti).',
 'NIfTI-MRS version 0.2',
 'Data shape (1, 1, 1, 4128, 32, 2)',
 "Dimension tags: ['DIM_COIL', 'DIM_EDIT', None]",
 'Spectrometer Frequency: 123.255794 MHz',
 'Dwelltime (Bandwidth): 3.333E-04s (3000 Hz)',
 'Nucleus: 1H',
 'Field Strength: 2.89 T',
 '']

## Post-process
### Coil combine

In [29]:
%sx fsl_mrs_proc coilcombine --file scan_1_68ms_2s/raw_nifti/metab_raw.nii.gz --reference raw_nifti/wref_internal.nii.gz --output scan_1_68ms_2s/processed --filename combined --overwrite -r
%sx fsl_mrs_proc coilcombine --file scan_1_68ms_2s/raw_nifti/wref_internal.nii.gz  --output scan_1_68ms_2s/processed --filename combined_wref
%sx fsl_mrs_proc coilcombine --file scan_1_68ms_2s/raw_nifti/wref_quant.nii.gz  --output scan_1_68ms_2s/processed --filename combined_quant

[]

### Align averages

In [20]:
%sx fsl_mrs_proc align --file scan_1_68ms_2s/processed/combined.nii.gz --output scan_1_68ms_2s/processed --filename aligned -r

[]

In [24]:
%sx fsl_mrs_proc align-diff --file scan_1_68ms_2s/processed/aligned.nii.gz --dim DIM_DYN --dim_diff DIM_EDIT --diff_type sub --output scan_1_68ms_2s/processed --filename aligned2 -r

[]

### Combine averages, ecc, phase, and Difference

In [31]:
%sx fsl_mrs_proc average --file scan_1_68ms_2s/processed/aligned2.nii.gz --dim DIM_DYN --output scan_1_68ms_2s/processed --filename avg

[]

In [56]:
%sx fsl_mrs_proc ecc --file scan_1_68ms_2s/processed/avg.nii.gz --reference processed/combined_wref.nii.gz --output scan_1_68ms_2s/processed --filename ecc -r
%sx fsl_mrs_proc truncate --file scan_1_68ms_2s/processed/ecc.nii.gz --points -2 --pos first --output scan_1_68ms_2s/processed --filename truncated -r
%sx fsl_mrs_proc phase  --file scan_1_68ms_2s/processed/truncated.nii.gz --ppm 2.8 3.1 --output scan_1_68ms_2s/processed --filename phased -r

[]

In [57]:
%sx fsl_mrs_proc subtract --file scan_1_68ms_2s/processed/phased.nii.gz --dim DIM_EDIT --output scan_1_68ms_2s/processed --filename diff -r

[]

In [62]:
%sx fsl_mrs_proc fixed_phase --file scan_1_68ms_2s/processed/diff.nii.gz --p0 195 --output scan_1_68ms_2s/processed --filename final_metab -r

[]

### Create ON/OFF Spectra

In [20]:
%sx mrs_tools split --file scan_1_68ms_2s/processed/phased.nii.gz --dim DIM_EDIT --index 1 --output scan_1_68ms_2s/processed --filename split
%sx mv scan_1_68ms_2s/processed/split_1.nii.gz scan_1_68ms_2s/processed/off.nii.gz
%sx mv scan_1_68ms_2s/processed/split_2.nii.gz scan_1_68ms_2s/processed/on.nii.gz

[]

### Process the quantification spectra

In [35]:
%sx mrs_tools split --file scan_1_68ms_2s/processed/combined_quant.nii.gz --dim DIM_EDIT --index 1 --output scan_1_68ms_2s/processed --filename quant_split
%sx mv scan_1_68ms_2s/processed/quant_split_1.nii.gz scan_1_68ms_2s/processed/quant.nii.gz
%sx rm scan_1_68ms_2s/processed/quant_split_2.nii.gz
%sx fsl_mrs_proc truncate --file scan_1_68ms_2s/processed/quant.nii.gz --points -2 --pos first --output scan_1_68ms_2s/processed --filename quant
%sx fsl_mrs_proc ecc --file scan_1_68ms_2s/processed/quant.nii.gz --reference scan_1_68ms_2s/processed/quant.nii.gz --output scan_1_68ms_2s/processed --filename quant

# %sx fsl_mrs_proc phase --file scan_1_68ms_2s/processed/quant.nii.gz --ppm 3 7 --output scan_1_68ms_2s/processed --filename quant -r

[]

### Fit

In [37]:
%sx fsl_mrs --data scan_1_68ms_2s/processed/final_metab.nii.gz\
    --basis uzay_svs_mpress_68_res_60_diff/\
    --keep NAA NAAG GABA Gln Glu GSH\
    --add_MM_MEGA\
    --ppmlim 0.2 4.2\
    --algo MH\
    --internal_ref NAA\
    --baseline_order -1\
    --h2o scan_1_68ms_2s/processed/quant.nii.gz\
    --output scan_1_68ms_2s/fit_diff\
    --report --overwrite

[]

In [36]:
%sx fsl_mrs --data scan_1_68ms_2s/processed/off.nii.gz\
    --basis uzay_svs_mpress_68_res_60_off/\
    --add_MM\
    --ppmlim 0.2 4.2\
    --algo Newton\
    --baseline_order 1\
    --h2o scan_1_68ms_2s/processed/quant.nii.gz\
    --output scan_1_68ms_2s/fit_off\
    --report --overwrite

[]

In [68]:
%sx fsl_mrs --help

['usage: fsl_mrs [-h] [-v] --data <str> --basis <str> --output <str>',
 '               [--algo ALGO] [--ignore METAB [METAB ...]]',
 '               [--keep METAB [METAB ...]] [--combine METAB [METAB ...]]',
 '               [--ppmlim LOW HIGH] [--h2o H2O] [--baseline_order ORDER]',
 '               [--metab_groups METAB_GROUPS [METAB_GROUPS ...]] [--add_MM]',
 '               [--add_MM_MEGA] [--lorentzian]',
 '               [--ind_scale IND_SCALE [IND_SCALE ...]] [--disable_MH_priors]',
 '               [--t1 IMAGE] [--TE TE] [--TR TR]',
 '               [--tissue_frac WM GM CSF OR json [WM GM CSF OR json ...]]',
 '               [--internal_ref INTERNAL_REF [INTERNAL_REF ...]]',
 '               [--wref_metabolite WREF_METABOLITE] [--ref_protons REF_PROTONS]',
 '               [--ref_int_limits REF_INT_LIMITS REF_INT_LIMITS]',
 '               [--h2o_scale H2O_SCALE] [--report] [--verbose] [--overwrite]',
 '               [--conj_fid] [--no_conj_fid] [--conj_basis] [--no_conj_basis